In [1]:
# 学习：通过接收到的数据，归纳提取相同与不同。
# 机器学习：让计算机以数据为基础，进行归纳与总结
# 模型：数据解释现象的系统
# 机器学习：
    # 监督学习（有标注）
        # 分类：标注是离散值
        # 回归：标注是连续值
    # 非监督学习（无标注）完全让数据自己说话，将数据不同的特征在不同的模型中进行不同的表现
        # 聚类分析
        # 关联分析
    # 半监督学习（部分有标注部分无标注）有标注的数据可以作用与无标注的数据，反过来也可以

In [2]:
# 一份数据集可分为：(所占的比例为6:2:2)
# 1.训练集 (6)：用来训练和拟合模型
# 2.验证集 (2)：当通过训练集训练出多个模型后，使用验证集数据纠偏或比较预测
# 3.测试集 (2)：模型泛化能力的考量。
    # 泛化：对未知数据的预测能力
# 如果没有验证集，则训练集和测试集的比例为4:1
# K-fold交叉验证：将数据集分成K份，每份轮流做一遍测试集，其它做训练集

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

In [5]:
# 归一化将数据转化到 0-1 之间的范围
# sl:satisfaction_level
# le:last_evaluation
# npr:number_project
# amh:average_monthly_hours
# tsc:time_spend_company
# wa:Work_accident
# pl5:promotion_last_5years
# False:MinMaxScaler
# True:StandardScaler
# 将离散值数值化
# dp:department 
# slr:salary
# False:LabelEncoding
# True:OneHotEncoder
# 降维
# lower_d = False  不降维
# ld_n = 1 下降的维度
def hr_preprocessing(sl=False, le=False, npr=False, amh=False, tsc=False, wa=False, pl5=False, dp=False, slr=False, lower_d = False, ld_n = 1):
    df = pd.read_csv('./data/HR.csv')
    # 1.清洗数据
    df = df.dropna(subset=['satisfaction_level', 'last_evaluation'])
    df = df[df['satisfaction_level']<=1][df['salary']!='nme']
    # 2.得到标注
    label = df["left"]
    df = df.drop('left', axis=1)
    # 3.特征选择
    
    # 4.特征处理
    scaler_lst = [sl, le, npr, amh, tsc, wa, pl5]
    column_lst = ['satisfaction_level', 'last_evaluation', 'number_project', 'average_monthly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years']
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            df[column_lst[i]]=MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1, 1)).reshape(1, -1)[0]
        else:
            df[column_lst[i]]=StandardScaler().fit_transform(df[column_lst[i]].values.reshape(-1, 1)).reshape(1, -1)[0]
    # # 将离散值数值化
    scaler_lst = [dp, slr]
    column_lst = ["department", "salary"]
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            if column_lst[i] == "salary":
                df[column_lst[i]] = [map_salary(s) for s in df['salary'].values]
            else:
                df[column_lst[i]] = LabelEncoder().fit_transform(df[column_lst[i]])
            df[column_lst[i]] = MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1, 1)).reshape(1, -1)[0]
        else:
            # OneHotEncoder
            df = pd.get_dummies(df, columns=[column_lst[i]])
    
    # 降维
    if lower_d:
        # n_components 不能大于类的个数
        # return LinearDiscriminantAnalysis(n_components=ld_n)
        
        # PCA n_components 不受限制
        return PCA(n_components=ld_n).fit_transform(df.values)
        
    
    return df, label

d = dict([('low', 0), ('medium', 1), ('high', 2)])
def map_salary(s):
    return d.get(s, 0)

# 训练集 验证集 测试集
def hr_modeling(features, label):
    # 引入切分训练集和测试集的函数
    from sklearn.model_selection import train_test_split
    # 先取值
    f_v = features.values
    l_v = label.values
    # test_size 测试集占多少比例  
    # X_tt  训练集部分 X_validation标注
    # y_tt  验证集部分 Y_validation标注
    # 得到验证集
    X_tt, X_validation, Y_tt, Y_validation = train_test_split(f_v, l_v, test_size=0.2)
    # 区分验证集和测试集
    X_train, X_test, Y_train, Y_test = train_test_split(X_tt, Y_tt, test_size=0.25)
    print(len(X_train), len(X_validation), len(X_test))
    
def main():
    # print(hr_preprocessing(lower_d = False, ld_n = 3))
    features, label = hr_preprocessing()
    hr_modeling(features, label)
if __name__=="__main__":
    main()

8999 3000 3000
